## 2.1. 학습 데이터
- AI 모델은 학습한 데이터의 특성에 따라 할 수 있는 일 결정
- 대규모 모델 학습에 필요한 데이터를 충분히 확보하는 것은 어렵고 비용이 많이 듦
- 비영리 단체가 인터넷 웹사이트를 주기적으로 크롤링해서 만드는 커먼 크롤 (Common Crawl)이 많이 사용됨
  - 데이터 품질이 의심스러움
  - 저품질을 걸러내기 위해 휴리스틱 사용하기도 함
- 더 많은 데이터로 학습하기 위해서는 더 많은 컴퓨팅 자원이 필요하고, 데이터가 많다고 모델 성능이 좋아지는 것음 아님
- 모델이 원하는 작업을 제대로 수행하기 위해서는 특정 요구사항에 맞는 데이터셋 선별이 중요

### 2.1.1. 다국어 모델
- 인터넷에서 가장 많이 사용되는 언어는 영어임
  - 이에 따라 범용 모델이 다른 언어보다 영어에서 더 좋은 성능 보임
- 반대로 AI 학습시키는 데 사용할 수 있는 데이터가 부족한 언어를 저자원 언어라고 함
  - 데이터가 적게 포함된 언어에 대해 성능 저하가 관찰됨
  - 이외에도 언어마다 고유한 구조와 문화적 특성이 존재하는데, 모델이 이 부분을 학습하기 어려워 성능이 저하되었을 가능성도 있음
- LLM이 일반적으로 번역을 잘하지만, 질의를 영어로 번역한 후 응답을 다시 원어로 번역하는 방식은 완벽한 해결책이 될 수 없음
  - 데이터가 적은 언어를 충분히 이해해 번역할 수 있는 모델이 필요
  - 번역 과정에서 정보가 손실될 수 있음
- 모델의 추론 지연 시간과 비용은 입력과 응답에 포함된 토큰 수에 비례하고, 언어별로 토큰화 효율성이 다르기 때문에, 영어가 아닌 언어 모델이 더 느리고 비용이 많이 들 수 있음
- 이외에도 예상하지 못한 성능 문제가 발생할 수 있음

### 2.1.2. 도메인 특화 모델
- 도메인 특화 작업에서 좋은 성능을 내는 모델을 학습하려면, 해당 분야의 전문적인 데이터셋이 필요

## 2.2. 모델링
- 모델 학습 전 모델을 어떻게 설계할지 결정해야 함
- 이에 따라 모델의 능력뿐만 아니라 모델을 활용해 만드는 서비스의 사용성에도 영향을 미침

### 2.2.1. 모델 아키텍처
- 오늘날 언어 기반 파운데이션 모델에서 가장 널리 쓰이는 아키텍쳐는 트랜스포머 아키텍처

#### 트랜스포머 아키텍처
- 기계 번역과 요약 성능을 크게 개선한 seq2seq의 문제점을 개선하기 위해 등장
- RNN (순환 신경망)
  - 시간 순서대로 주어지는 학습 데이터 집합을 사용하여 지도학습을 수행하는 모델
  - 수식
    - 일반 신경망의 수식
      - $$y=f(W\cdot x+b)$$
    - 출력층
      - $$y_{t}=f(W_{hy}h_{t}+b_{y})$$
      - $W_{hy}$
        - 은닉 상태에서 출력층으로 가는 가중치
      - $f$ 
        - 활성화 함수
        - 분류 문제의 경우 주로 Softmax 사용
    - 은닉 상태
      - $$h_{t}=\tanh (W_{xh}x_{t}+W_{hh}h_{t-1}+b_{h})$$
      - $W_{xh}x_{t}$
        - 현재 입력에 대한 가중치
      - $W_{hh}h_{t-1}$
        - 이전 상태에 대한 가중치 (기억)
      - $tanh$: 활성화 함수
        - 주로 tanh 또는 ReLU 사용
  ![rnn](../assets/images/2-1.rnn.png)
- seq2seq
  - 입력과 출력은 모두 토큰 시퀀스
  - 입력을 처리 (벡터화)하는 인코더와 출력을 생성하는 디코더로 구성
  - 인코더와 디코더로 RNN 사용
  - 인코더는 입력 토큰을 순차적으로 처래해 입력을 표현하는 최종 은닉 상태 (hidden state)를 출력
  - 디코더는 최종 은닉 상태와 이전에 생성된 토큰을 기반으로 출력 토큰을 순차적으로 생성
  - 문제점
    - seq2seq 디코더의 입력의 최종 은닉 상태만 사용해 출력 토큰을 생성
      - 책 요약본 기반으로 응답을 만드는 것과 비슷해 출력물의 품질이 떨어짐
      - **트랜스포머는 각 출력 토큰 생성 시 서로 다른 입력 토큰의 중요도에 가중치를 둠으로써 책의 전체 페이지를 참조해 응답을 생성하는 효과를 냄**
    - 입력 처리와 출력 생성이 모두 순차적으로 이루어져야 해서 시퀀스가 길어질수록 속도가 느려짐
      - **트랜스포머는 RNN 사용하지 않고 설계되어 입력 토큰을 병렬로 처리해 입력 처리 속도 크게 향상**
      - 자기회귀 언어 모델은 순차 출력의 병목 현상은 남아 있음
  ![seq2seq vs transformer](../assets/images/2-2.seq2seq-versus-transformer.png)
- 트랜스포머 기반 언어 모델의 추론
  1. 프리필
    - 입력 토큰을 병렬로 처리해 첫 번째 출력 토큰 생성하는 데 필요한 중간 상태를 만듦
    - 각 입력 토큰의 키 벡터와 값 벡터가 중간 상태에 저장
  2. 디코드
    - 출력 토큰을 한 번에 하나씩 생성

#### 어텐션 메커니즘
- 트랜스포머 모델 작동 방식을 이해햐기 위해서는 어텐션 메커니즘을 이해해야 함
- 어텐션 메커니즘은 내부적으로 키 (key), 값 (value), 쿼리 벡터 (query vector)를 활용
  - 인코더의 마지막 층에서 나온 벡터에 서로 다른 가중치 행렬을 곱 (사영, projection)해 같은 데이터를 서로 다른 각도 (공간, 구체적으로 검색 기준 공간 (query space), 검색 색인 공간 (key space), 정보 전달 공간 (value space))을 바라보개 만듦
  - **하나의 벡터에 부여하는 역할을 분리함으로써 모델 학습을 용이하게 해 표현력 증가**
- 쿼리 벡터
  - 각 디코딩 단계에서 디코더의 상태를 나타냄
  - **찾고 싶은 정보의 방향**
  - e.g. "나는 지금 주어를 찾고 있어", "나는 목적어가 필요해", "나는 시간 정보를 찾고 있어"
- 키 벡터
  - 이전 토큰을 나타냄
  - **어떤 정보를 갖고 있는지의 색인**
  - e.g. "나는 주어 역할을 할 수 있음", "나는 시간 표현임", "나는 동사임"
- 값 벡터
  - 모델이 학습한 이전의 토큰의 실제 값
  - **실제 전달된 정보**
  - e.g. "이 토큰이 가진 실제 의미 벡터"
- 쿼리 벡터와 키 벡터 간의 **내적을 통해 각 입력 토큰에 얼마나 주목할지 계산**
  - 내적은 기하학적으로 $\vec{a} \cdot \vec{b} = \|\vec{a}\| \|\vec{b}\| \cos\theta$
    - $\theta$ = 두 벡터 사이의 각도
    - $\cos\theta$ = 방향 유사도
    - 방향 정렬 정도, 크기 정보를 통해 두 벡터 간 유사도 측정 가능
- 쿼리, 키, 값 행렬의 차원은 모델의 은닉 차원에 해당
- 이전 토큰마다 키 벡터와 값 벡터가 있기 때문에, 시퀀스가 길어질수록 더 많은 키 벡터와 값 벡터를 계산하고 저장해야 함
- 대부분 **멀티 헤드**로 구현
  - 모델이 서로 다른 이전 토큰 그룹을 동시에 주목할 수 있음
  - 쿼리, 키, 값 벡터가 더 작은 벡터로 나뉘어 각각 하나의 어텐션 헤드로 할당
  - 모든 어텐션 헤드의 출력은 이어 붙여져 출력 투영 행렬을 통과한 뒤 다음 연산 단계로 넘어감
  - 트랜스포머에서 멀티헤드는 히든 레이어 단에서 적절한 가중치를 곱해 기존 행렬 대비 차원이 낮은 서브 벡터로 서로 다른 공간에 투사함으로써 **풍부한 표현력을 갖게 함**
    - e.g. 
      - 헤드 1 (문법적 관점): "이 주어(Q)에 대응하는 동사(K)는 어디 있지?"
      - 헤드 2 (대명사 해소): "여기 있는 '그(He, Q)'가 가리키는 실제 인물(K)은 누구지?"
      - 헤드 3 (수식 관계): "이 형용사(Q)가 수식하는 명사(K)는 무엇이지?"
  - 다른 토큰 그룹에 주목하게 한다는 의미는 결과론적으로 **다른 관점으로 유사도를 측정**함으로써 헤드별로 다른 토큰에 주목하게 한다는 뜻

#### 트랜스포머 블록
- 트랜스포머 아키텍처는 여러 개의 트랜스포머 블록으로 구성
- 각 트랜스포머 블록은 **어텐션 모듈과 다층 퍼셉트론 (multi-layer perceptron, MLP) 모듈을 포함**
- 트랜스포머 블록 수는 흔히 모델 레이어 수

##### 어텐션 모듈
- 쿼리, 키, 값, 출력 투영이라는 네 가지 가중치 행렬로 구성

##### MLP 모듈
- 비선형 함수로 구분된 선형 레이어로 구성 
- 선형 레이어는 선형 변환에 사용되는 가중치 행렬
  - 피드포워드 레이어라고도 함
    - 피드포워드 레이어는 입력이 한 방향으로만 흐르는 신경망을 뜻함
- **어텐션이 '어떤 토큰을 볼 것인가'를 결정한다면, MLP는 '그 토큰들을 보고 난 뒤, 내(현재 토큰) 의미를 어떻게 업데이트할 것인가'를 처리하는 내부 재가공 공정**

##### 트랜스포머 블록 이전의 임베딩 모듈
- **토큰을 임베딩 벡터로 변환하는 임베딩 행렬과 토큰 위치를 임베딩 벡터로 변환하는 위치 임베딩 행렬로 구성**
- 최종적으로 두 벡터를 합산
- 위치 색인의 수가 모델 최대 컨텍스트 길이를 결정
  - 위치 색인 수 늘리지 않고 모델 컨텍스트 길이 늘릴 수 있는 기법 존재

##### 트랜스포머 블록 이후의 출력 레이어
- **모델 출력 벡터를 모델 출력 샘플링하는 데 사용되는 토큰 확률로 매핑**
- 언임베딩 레이어라고 하는 하나의 행렬로 구성
- 출력 레이어를 모델 헤드라고 부름

![transformer architecture](../assets/images/2-3.transformer-architecture.png)

#### 다른 모델의 아키텍처
- 트랜스포머보다 뛰어난 새로운 아키텍처를 개발하는 것은 쉽지 않음
- 트랜스포머 외 인기 있는 모델 중 하나는 RWKV (**R**eceptance + **W**eighted **K**ey-**V**alue)
  - 병렬로 학습할 수 있는 RNN 기반 모델
  - 컨텍스트 길이 제한이 없음
  - 긴 컨텍스트에서 좋은 성능이 보장되지는 않음
  - Attention을 RNN처럼 동작하도록 재구성한 모델
    - “전역 비교” 대신 “요약된 상태(state)”와 상호작용한다는 뜻
- 긴 시퀀스를 모델링하는 데 큰 잠재력을 보여준 아키텍처로는 SSM(**S**tate **S**pace **M**odel)이 있음
  - 행렬 연산을 통해 “이전 상태(기억)”를 업데이트하는 모델
- 모델 기본 아키텍처가 바뀌더라도 근본적인 접근 방식은 변하지 않을 것 

| 비교 항목 |	Transformer |	RWKV |	SSM (Mamba) |
| ---------- | ----------- | ---- | ----- |
| 메커니즘 |	전역 비교 (Attention)	| 선형 어텐션 (RNN식) |	상태 공간 모델 (Selective) |
| 정보 참조 |	과거 이력 전체 참조	| 압축된 상태 업데이트 |	압축된 상태 업데이트 |
| 추론 효율 |	낮음 (복잡도)	| 높음 (고정 메모리) |	높음 (고정 메모리) |
| 학습 병렬화 |	매우 우수 |	우수 |	우수 |
| 핵심 장점 |	정확한 장기 기억 |	효율적인 RNN형 구조 |	매우 긴 맥락 처리 최적화 |

### 2.2.2. 모델 크기
- 파운데이션 모델에서 파라미터 수가 중요하며, 보통 모델 이름 뒤 표시
  - 모델 파라미터 수가 늘어나면 학습 용량이 커져 성능 향상
  - 파라미터 수는 모델 학습하고 실행하는 데 필요한 컴퓨팅 자원을 추정하는 데 도움
  - 파라미터 중 0인 값이 높은 비율을 차지하는 (희소한) 경우 더 효율적인 데이터 저장과 연산이 가능
    - e.g. 일부 파라미터 그룹만 활성화되는 전문가 혼합 (mixture of experts, MoE)
- 모델 크기를 논할 때 데이터 크기 고려해야 함
  - 충분한 데이터로 학습하지 않는 경우 더 작은 모델보다 성능 떨어짐
  - 데이터셋 크기를 학습 데이터 수로 측정 가능
    - 데이터마다 정보량이 다르기 때문에 적절하지 않음
  - 데이터셋에 포함된 토큰의 개수를 세는 것이 보다 적절
    - 모델이 토큰 단위로 동작
    - 모델마다 토큰화 방식이 달라 완벽한 기준은 아님 
    - 데이터셋의 토큰 수와 학습 토큰 수는 다름
      - 학습 토큰 수는 실제 학습한 토큰의 총량을 의미
- 대규모 모델 학습 시 많은 컴퓨팅 자원을 필요로 함
  - GPU, CPU, TPU 같은 기기 수로써 필요 컴퓨팅 자원 측정
    - 기기마다 성능과 비용 크게 다름
  - 좀더 표준화된 단위는 부동소수점 연산 (floating point operation, FLOP)
    - 작업 수행하는 데 필요한 부동소수점 연산의 개수
    - 기계의 최대 처리 성능을 측정하는 초당 부동소수점 연산을 뜻하는 FLOP/s와 구분
  - 활용률은 기기의 최대 컴퓨팅 성능 대비 실제 사용할 수 있는 비율을 뜻함
    - 좋은 활용률은 모델, 작업 유형, 하드웨어에 따라 다르지만, 일반적으로 50% 정도를 괜찮은 수준으로 봄  

#### 스케일링의 법칙: 컴퓨팅 자원 최적화 모델 만들기
- 컴퓨팅 - 최적 모델 (compute-optimal model)
  - **고정된 컴퓨팅 예산 내 최고 성능**을 달성할 수 있는 모델
- 친칠라 스케일링 법칙
  - 컴퓨팅 예산이 주어졌을 때 최적의 모델 크기와 데이터셋 계산하는 규칙
  - 컴퓨팅 최적 학습을 위해서는 학습 토큰 수가 모델 크기의 약 20배여야 함
- 컴퓨터 - 최적 계산은 데이터 획득 비용이 컴퓨팅 비용보다 저렴하다는 가정에 기반
- 특정 모델 성능 달성하는 비용은 감소 중
- 모델 성능 향상에 드는 비용은 여전히 큼
- 실제 제품은 모델 품질 외 사용성 등도 중요

##### 스케일링 외삽
- 하이퍼파라미터 vs 파라미터
  - 파라미터
    - 학습 과정 중 모델에 의해 학습되는 값 
  - 하이퍼라파라미터
    - 사용자가 모델을 구상하고 모델의 학습 방식을 제어하기 위해 설정하는 값 
  - e.g.
    - 하이퍼파라미터: Thompson Sampling에서의 alpha prior, beta prior
    - 파라미터: Thompson Sampling에서의 alpha, beta
- 모델 성능은 하이퍼파라미터 값에 의해 좌우
- 작은 모델에서는 서로 다른 하이퍼파라미터 조합으로 모델 여러 번 학습하고 가장 좋은 성능을 보이는 것을 일반적
- 큰 모델은 자원 소모가 커 위와 같은 방식 사용하기 어려움
- 스케일링 외삽 (하이퍼파라미터 전이)
  - 외삽 (Extrapolation)
    - 학습(또는 관측)한 범위를 벗어난 구간에 대해 예측하는 것
  - 다양한 크기의 모델에서 하이퍼파라미터가 미치는 영향을 연구하고, 목표 크기에서 어떻게 작동할지 외삽
  - 아래의 이유로 연구가 어려움
    - 대규모 모델 학습 연구한 경험과 자원을 가진 사람이 거의 없음
    - 하이퍼파리미터 수 많고 서로 상호작용하는 방식
    - 창발력
      - 규모가 커져야 나타나는 능력

##### 스케일링 병목 현상 
- 학습 데이터셋
  - 학습 데이터셋 크기의 증가율이 새로 생성되는 데이터의 증가율보다 큼
  - 공개적으로 이용 가능한 데이터 소진 시 독점 데이터 활용 가능
- 전기

## 2.3. 사후 학습
- 사전 학습 모델의 문제점
  - 대화가 아닌 텍스트 완성을 잘하도록 학습
  - 인터넷에서 무차별적으로 수집된 데이터로 학습 시 출력물이 인종 차별적이거나, 성차별적이거나, 무례하거나, 틀린 답일 수 있음
- 위의 문제를 해결하기 위해 사후 학습 진행
- 사후 학습은 사전 학습에 비해 적은 양의 자원 소비
- 일반적으로 두 단계로 구성
  - 지도 파인튜닝 (SFT)
  - 선호도 파인튜닝

### 2.3.1. 지도 파인튜닝
- 완성이 아닌 대화를 위해 모델 최적화
- 행동 복제라고도 함
  - 어떻게 행동해야 하는지 보여주면 모델이 행동을 복제하기 때문
- 시연 데이터로 사전 학습된 모델 파인튜닝
  - 시연 데이터로 고품질 사람 데이터 사용
  - 비용 문제로 휴리스틱을 사용하거나 AI 생성 데이터를 검토하기도 함
    - 파인튜닝 대신 처음부터 시연 데이터로 학습시킬 수 있으나 사전 학습 방식이 성능이 더 좋은 경우가 많음

### 2.3.2. 선호도 파인튜닝
- 보편적인 사람의 선호도가 존재하며, 이를 AI에 내장할 수 있다고 가정
- 사람의 선호도에 맞는 응답 출력하도록 모델 파인튜닝
- 사람 피드백 기반 강화 학습 (RLHF)이 가장 널리 쓰임
  - 아래 두 부분으로 구성
    - 파운데이션 모델의 출력에 점수를 매기는 보상 모델
    - 보상 모델이 최대 점수를 줄 응답을 생성하도록 모델을 최적화
- 이외에도 직접 선호도 최적화 (DPO), AI 피드백 기반 강화 학습 (RLAIF) 사용됨
  - RLHF가 DPO보다 복잡하지만 모델 조정할 수 있는 더 많은 유연성 제공

#### 보상 모델
- RLHF 보상 모델 기반으로 동작
- (프롬프트, 응답) 쌍이 주어지면 그 응답이 얼마나 좋은지 점수 매겨 학습
  - 각 샘플을 독립적으로 평가하는 것을 포인트와이즈 평가라고 함
  - 이때 데이터의 신뢰도가 중요한데, 포인트와이즈 평가에서 레이블러마다 혹은 같은 레이블러가 여러 번 평가할 때마다 같은 샘플에  대해 다른 점수를 줄 수 있음
- 위의 이유로, AI가 여러 응답 생성하고 응답 간 선호도를 비교한, (프롬프트, 선호 응답, 비선호 응답) 형식의 비교 데이터를 사용
- 이때 선호 응답과 비선호 응답 간 출력 점수 차이를 나타내는 목적 함수로써 모델이 구체적인 점수를 매기도록 학습할 수 있음

#### 보상 모델을 사용한 파인튜닝
- SFT로 학습된 모델이 여러 질문에 답을 생성하고, 그 답을 보상 모델이 평가한 점수가 높아지도록 강화 학습 알고리즘인 PPO (Proximal Policy Optimization)를 통해 모델을 반복적으로 업데이트
  - PPO는 보상을 최대화하도록 정책(토큰 확률 분포)을 업데이트하되, 기존 정책에서 너무 멀어지지 않도록 확률 비율을 클리핑하여 안정적으로 학습하는 강화학습 알고리즘
- SFT만 사용할 때보다 RLHF나 DPO를 사용할 때 성능이 더 좋아짐
- 일부 강화 학습을 뛰어넘고 모델이 생성한 여러 샘플 중에 가장 좋은 것을 선택 (Best of N 전략)하기도 함
- 지도 파인튜닝과 선호도 파인튜닝 모두 사전 학습에 사용된 데이터 품질이 낮아서 생긴 문제를 해결하기 위한 단계로, 더 좋은 사전 학습 데이터나 파운데이션 모델 학습 기법이 개발된다면 필요하지 않을 수 있음

## 2.4. 샘플링
- 샘플링 과정을 통해 출력 생성
- 샘플링 전략과 변수로 온도 (temperature), top-k, top-p가 있음
- 샘플링은 AI 출력을 확률적으로 만들고, 이는 AI의 동작을 다루는 데 중요

### 2.4.1. 샘플링의 기초
- 입력이 주어지면 신경망은 잠재적인 결과들의 확률을 먼저 계산해 출력 생성
- 언어 모델의 경우 다음 토큰을 생성하기 위해 어휘들의 모든 토큰에 대한 확률 분포 계산
  - 입력이 주어지면 로짓 벡터 출력
  - 각 로짓은 하나의 잠재적인 값으로, 모델 어휘집에 있는 하나의 토큰
  - 로짓 벡터의 크기는 어휘집의 크기와 같음
  - 더 큰 로짓이 더 높은 확률에 대응하나, 로짓은 확률을 나타내지 않음
    - 로짓은 음수가 될 수 있으며, 로짓의 합은 1이 되지 않음
  - 로짓을 확률로 변환하기 위해 소프트맥스 레이어 자주 사용
    - 소프트맥스 레이어는 스트프맥스 함수를 적용하는 레이어를 의미
    - 소프트맥스 함수는 0 - 1 사이의 출력값을 가지며, 출력값의 합이 1이 됨 
    - $$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$$
- 잠재적인 결과들의 확률이 서로 다를 때 가장 높은 결과를 선택하는 것을 그리디 샘플링이라고 함
  - 언어 모델의 경우 그리디 샘플링 사용 시 지루한 출력을 만들 수 있음

### 2.4.2. 샘플링 전략
- 적절한 샘플링 전략 사용 시 더 적합한 응답 생성하도록 만들 수 있음
- 모델 로짓에 접근 가능할 경우 샘플링 전략을 설계해 특정 속성을 가진 응답 유도할 수 있음

#### 온도 
- 온도는 소프트맥스 변환 전 로짓을 조정하는 데 사용되는 함수
- 온도가 높을수록 모델이 가장 명백한 값 (가장 높은 로짓을 가진 값)을 선택할 가능성이 낮아져, 모델 출력이 더 창의적이지만 일관성이 떨어질 수 있음
- 온도가 낮을수록 모델이 가장 명백한 값을 선택할 가능성이 높아져, 모델의 출력이 더 일관적이지만 더 지루해질 수 있음
- 모델 제공업체들은 일반적으로 온도를 0에서 2로 제한
  - 기술적으로 0으로 로짓을 나눌 수 없어 온도 값은 0이 될 수 없음
  - 온도 0으로 설정 시 모델은 로짓 조정과 소프트맥스 계산하지 않고 가장 큰 로짓 값 가진 토큰 선택
- 모델 제공업체들은 모델이 생성한 확률을 로그 스케일로 표현된 로그프롭 (logprob)으로 제공
  - 숫자가 주어진 형식으로 표현하기에는 너무 작아서 0으로 내림되는 언더플로우 문제를 줄일 수 있기 때문
  - 모델 내부 작동을 이해하는데 유용하나, 노출하지 않거나 제한적으로만 노출

#### top-k
- 모델 응답 다양성을 너무 희생하지 않으며 계산 작업량을 줄이기 위한 샘플링 전략
- 소프트맥스 함수는 모든 후보 값에 대해 지수합 ($\sum_{j=1}^{n} e^{z_j}$)과 $\frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$ 계산
- 위의 계산 비용을 줄이기 위해, 로짓 계산 후 상위 k개 선택 후 상위 k개 로짓에 대해서만 소프트맥스 수행
- 목표로 하는 응답의 다양성에 따라 5에서 500 사이 값 사용
- k 작아지면 모델이 선택할 수 있는 단어의 수가 줄어들어 더 예측 가능하고 덜 흥미로운 텍스트가 만들어짐 

#### top-p
- 뉴클리어스 샘플링 (nucleus sampling) 이라고 알려짐
- 가장 가능성 높은 다음 값의 확률을 내림차순으로 합산하고 합이 p에 도달하면 중단함으로써, 샘플링할 값을 동적으로 선택할 수 있음
- 일반적으로 p는 0.9에서 0.95 사이
- 소프트맥스 계산을 부하를 반드시 줄여주지 않음
- 컨텍스트와 관련 높은 단어들만 후보로 삼아 컨텍스트에 적절한 문장을 생성할 수 있음
- 관련 샘플링 전략으로 고려 대상이 될 토큰의 최소 확률을 설정하는 min-p가 있음

#### 중단 조건
- 긴 출력 시퀀스는 더 많은 시간과 계산을 필요로 하고 사용자에게 부정적인 경험을 줄 수 있어, 모델이 시퀀스 생성을 중단하는 조건 설정이 필요
- 일정 개수의 토큰을 생성한 후에 자동으로 멈추도록 설정 가능
  - 문장 중간에 잘릴 가능성이 있음
- 중단 토큰이나 중단 단어 사용
- 중단 조건으로 특정 형식으로 출력 생성 시 조기 중단으로 출력 형식이 잘못되리 수 있음

### 2.4.3. 테스트 시점 연산
- 좋은 응답 나올 확률을 높이기 위해 여러 응답 생성
- 테스트 시점은 추론 시점을 뜻하며, 샘플링 가능한 출력 수가 매 추론 호출 시 할당할 수 있는 연산 자원의 양에 따라 결정되기 때문에 테스트 시점 연산으로 명명
- 무작위로 여러 출력 생성하고 가장 적합한 출력 선택하는 best of N 기법, 시퀀스 생성 단계마다 가장 가능성 높은 후보 (빔)을 정해진 개수만큼 생성하는 빔 검색 등의 방법이 있음
- 최고의 출력을 선택하기 위해 아래 중 하나의 방법을 선택할 수 있음
  - 사용자가 선택하게끔 함
  - 확률이 가장 높은 출력으로 자동 선택
    - 이때 숏 시퀀스에 대한 편향을 피하기 위해 시퀀스의 합을 길이로 나눠 평균 로그프롭 사용 가능
  - 보상 모델을 사용해 각 출력의 점수를 매김
  - 검증기 사용
  - 휴리스틱 사용
- 비용이 많이 듦
- 테스트 시점 연산을 확장하는 것 (추론 중 더 많은 출력을 생성하기 위해 더 많은 계산을 할당)이 모델 파라미터 확장하는 것보다 효율적일 수 있음
  - OpenAI 실험에서 더 많은 출력을 샘플링하면 성능 향상되었으나, 그 효과는 400개의 출력까지만 지속
- 모델에게 여러 응답 병렬로 생성하도록 한 뒤 가장 먼저 완성되는 유효한 응답을 보여줘 지연 시간 문제를 극복할 수도 있음
- 여러 출력 중에서 가장 많이 나온 결과를 골라 정확한 답을 요구하는 작업 수행 가능
- 모델이 견고하지 않을수록 여러 출력 샘플링하는 게 유리
  - 입력이 조금 달라져도 출력이 크게 바뀌지 않을 때 모델이 견고하다고 봄

### 2.4.4. 구조화된 출력
- 아래 두 가지 경우 모델이 특정 형식을 따르는 출력을 생성해야 함
  - 시맨틱 파싱
    - 자연어를 기계가 읽을 수 있는 구조화된 형식으로 변환하는 것을 포함
  - 다운스트림 애플리케이션에서 출력이 사용되는 작업
    - 출력이 다른 애플리케이션에서 활용될 때 해당 애플리케이션이 처리할 수 있는 형태여야 함
- 아래 방법으로 구조화된 출력을 생성하도록 유도
  - 이미 구조화된 출력을 잘 생성하지만 개선이 필요한 경우
    - 프롬프팅
    - 후처리
    - 테스트 시점 연산
  - 집중적인 처리가 필요한 경우 
    - 제약 샘플링
    - 파인튜닝

#### 프롬프팅
- 모델의 지시 수행 능력과 지시의 명확성에 따라 모델이 지시를 따를 수 있을지 결정
- 항상 지시를 따른다는 보장은 없음
- 유효 출력 비율을 높이기 위해 AI 평가자 접근 방식 사용
  - 프롬프트 출력을 검증하거나 수정하기 위해 AI 사용
  - 출력 생성과 검증을 위해 최소 두 번의 모델 질의가 필요하기 때문에 비용과 지연 시간이 늘어남

#### 후처리
- 간단하고 비용이 적게 들지만 큰 효과를 볼 수 있음
- 모델이 자주하는 실수를 파악해 수정하는 스크립트 작성
- 수정하기 쉬운 실수에만 효과적

#### 제약 샘플링
- 제약 조건을 만족하는 값들 중에서 토큰 샘플링해 제약 조건에 맞게 텍스트 생성을 유도하는 기법
- 각 단계에서 허용되는 것과 허용되지 않는 것을 지정하는 문법 필요
- 외부 도구에서 지원하거나 팀에서 자체적으로 개발한 문법이 있는 형식에만 사용 가능
- 문법 검증 때문에 생성 지연 시간이 증가할 수 있음
- 모델 학습에 투자하는 게 더 좋다는 의견도 있음

#### 파인튜닝
- 원하는 형식에 맞는 예시로 모델 학습
  - 가장 효과적이고 일반적
  - 기대 형식을 지켜 출력한다고 보장할 수 없음
- 특성 기반 전이
  - 모델 아키텍처를 수정해 출력 형식을 보장
  - 모델이 학습한 '지식(Feature)'은 그대로 가져오되 그 지식을 '어떻게 내보낼지(Output Head)'를 완전히 새로 설계하는 기법을 의미
- 전체 학습 시 많은 리소스가 필요하지만 성능은 좋아짐
- 기본적으로 모델만으로는 구조화된 출력을 생성할 수 없다고 가정하기 때문에, 모델이 강력해지면 단순한 지시로도 구조화된 출력 생성할 것으로 기대

### 2.4.5. AI의 확률적 특성
- 모델이 응답을 샘플링하는 방식은 확률적
  - 반대는 결정론적으로 무작위성 없이 결과가 하나로 정해지는 것을 의미
- 대규모 데이터셋으로 학습되기 때문에, 다양한 가능성이 담겨 있고, 0이 아닌 확률이라면 터무니 없거나 틀린 내용도 AI가 생성 가능
- 이 때문에 일관성이 떨어지고 환각 현상 발생
  - 환각은 사실에 근거하지 않는 응답을 하는 현상 의미

#### 비일관성
- 두 가지 상황에서 발생
  - 같은 입력, 다른 출력
    - 아래 방법으로 해결 가능하나 완전히 일관된 결과를 보장하진 않음
      - 같은 질의가 들어올 경우 같은 응답하도록 캐싱
      - 온도, top-p, top-k 값과 같은 모델 샘플링 변수 고정
      - 토큰 샘플링에 사용되는 난수 생성기 초기값인 시드 변수 고정
    - 출력 생성 하드웨어도 결과에 영향을 미칠 수 있기 때문
      - 출력 생성을 환경에 따라 고정할 수 있음
      - 다만 위의 방법이 시스템에 대한 신뢰를 주진 못함
  - 살짝 다른 입력, 완전히 다른 출력
    - 서로 다른 입력에 대해 같은 출력을 생성하도록 강제할 순 없음
    - 신중하게 만든 프롬프트와 메모리 시스템을 통해 원하는 응답에 가까운 결과 생성은 가능

#### 환각
- 생성 모델에 흔한 현상
- 원인은 샘플링 과정만으로 설명 불가
- **언어 모델이 주어진 데이터와 자신이 생성한 데이터를 구분하지 못해 발생**한다는 가설이 있음
  - 자기 기만 (self-delusion)
  - 눈덩이처럼 불어나는 환각
    - 잘못된 가정을 한 후 모델이 잘못된 가정을 정당화하기 위해 계속 환각 일으킴
  - 해결 방법
    - 강화 학습을 통해 사용자가 제공한 프롬프트 (세상에 대한 관찰)와 생성한 토큰 (모델의 행동)을 구분하도록 함
    - 지도 학습을 활용해 학습 데이터에 사실과 반사실적 데이터 포함
- **모델 내부 지식과 레이블러 내부 지식이 일치하지 않아 발생**한다는 가설도 있음
  - 지도 파인튜닝에서 모델은 레이블러를 모방하도록 학습하는데, 레이블러는 갖고 있지만 모델이 갖고 있지 않은 지식을 이용한다면 환각을 일으키도록 학습시키고 있는 것
  - LLM이 무엇을 알고 모르는지 스스로 판단할 수 있다고 믿기도 함
    - 모델이 아는 정보만으로 응답하도록 강제해 환각 해결할 수 있다고 봄
    - 구체적으로 아래 두 가지가 해결책으로 제안
      - 출처를 검색하도록 요청해 검증
      - 강화 학습 사용
        - RLHF가 환각을 줄인다는 발견도 있지만, 다른 연구에서는 다른 측면은 개선하나 환각을 증가시키는 것으로 나타남
- 위의 두 가지 가설은 상호보완적